# Abstract
#### Setting : 실험목적, 실험에 사용한 데이터셋 명시

                            1. 원시데이터인 VOC.xlsx  데이터를 사용
                            2. VOC 유형을 Label로 지정
                            3. 사용하지 않는  컬럼제거
                            5. 특수 문자 제거
                            6. Kospacing 라이브러리를 이용하여 1차 자동 띄어 쓰기 라이브러리 적용
                            7. 정교한 띄어쓰기를 위해 han-spell라이브러리를 이용하여 2차 자동띄어쓰기 적용 
                            8. 불용어 처리

                                                      
                           8666 rows × 4 columns

In [5]:
import pandas as pd
import sys

In [6]:
from collections import Counter

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os


import warnings
warnings.filterwarnings('ignore')
import timeit

In [7]:
from pykospacing import Spacing
spacing = Spacing()
spacing("김형호영화시장분석가는'1987'의네이버영화정보네티즌10점평에서언급된단어들을지난해12월27일부터올해1월10일까지통계프로그램R과KoNLP패키지로텍스트마이닝하여분석했다.")
"김형호 영화시장 분석가는 '1987'의 네이버 영화 정보 네티즌 10점 평에서 언급된 단어들을 지난해 12월 27일부터 올해 1월 10일까지 통계 프로그램 R과 KoNLP 패키지로 텍스트마이닝하여 분석했다."
# Apply a list of words that must be non-spacing
spacing('귀밑에서턱까지잇따라난수염을구레나룻이라고한다.')

'귀 밑에서 턱까지 잇따라 난 수염을 구레나 룻이라고 한다.'

In [8]:
data = pd.read_csv('../../VOCData.csv',names=['ID', 'A', 'B', 'C', 'D','E','F','G','H','I','j','세부내용','L','M'])

#print("train set: {}개".format(len(df)))

In [9]:
data = data.reset_index(drop=True)
df = data[:10000]

### 데이터 전처리

In [10]:
from pykospacing import Spacing
spacing = Spacing()

def processsing(df) :
    no_spacing_data = []
    # df_col = []
    # ## 필요 없는 문자 '호' 제거
    # for i in range(len(df)):
    #     romove_col = df['세부내용'][i].replace('호',"")
    #     df_col.append(romove_col)
    # df['세부내용'] = df_col

    ## 특수문자 제거
    df = df['세부내용'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ")

    df = pd.DataFrame({'세부내용':df})
    return df

### 1차 자동 띄어 쓰기 적용

In [11]:
## 띄어쓰기 로직을 데이터 셋에 적용
def kospacing(df) :
    global preprocess_data
    preprocess_data = None
    
    clean_data = []
    for row in df.iterrows() :
        
        spacings = spacing(row[1]['세부내용'])
        clean_data.append(spacings)

        preprocess_data = pd.DataFrame({'sentence':clean_data})
        
    return preprocess_data


### 2차 자동 띄어쓰기

In [12]:
from hanspell import spell_checker

## 띄어쓰기 로직을 데이터 셋에 적용
def han_spell(df) :
    clean_data = []
    for i in range(len(df)) :
        spelled_sent = spell_checker.check(df['sentence'][i])
        hanspell_sent = spelled_sent.checked
        clean_data.append(hanspell_sent)
        preprocess_data = pd.DataFrame({'sentence':clean_data})
        
    return preprocess_data


In [13]:
df = processsing(df)
df = kospacing(df)
# preprocess_data = han_spell(df)
# data = pd.read_excel('../../Common/data/VOC.xlsx', header=1)

# df = pd.concat([data['접수시간'], data['본부'], data['VOC 유형'], preprocess_data['sentence']],axis= 1)

In [ ]:
data = data[:10000]
df = pd.concat([ data['E'], data['I'], preprocess_data['sentence']],axis= 1)

In [ ]:
pd.set_option('display.max_rows', None)
df.head()

,E,I,sentence
0,구로지사,부팅중 오류메시지 발생,차 동 일 고객 요청사항 리셋 접수 호 수신 불 수신카드 오류 리셋 회선 단말특이사...
1,전주지사,장비리셋 후 재사용 요청,리셋 접수 호 채널 수신불량
2,부천지사,수신 불량,고객 요청사항 특정 체널 수신 불 번 안 나옴 재부팅하면 사용 가능하다가 다시 안됨...
3,용인지사,장비리셋 후 재사용 요청,리셋 접수 채널 수신 불량 인터넷 연결오류
4,천안지사,수신 불량,고객 요청사항 안됨 리셋 연락 후 회선 단말특이사항 진단 결과 기타 추가 연락처


### 불용어 처리

In [ ]:
stop_words = "일 고객 요청사항 독촉심함 독촉 봇 빠른 방문 요청  단말특이사항 불 후 정상 측 연락처 안 됨기사 추가 특이사항 서비스 회선 진단 결과 코로나 기타 리셋 안방 거실 독촉"

def stop_word(df) :
    global stop_words

    df = df.reset_index(drop= True)
    preprocess_data = []
    for i in range(len(df)) :
        preprocess_data.append(df['sentence'][i].split(' '))

    stop_words=stop_words.split(' ')

    df['words_list'] = 0

    result = []
    for sentence_words in preprocess_data:
        list_a = []
        for word in sentence_words:
            if word not in stop_words:
                list_a.append(word)
        result.append(list_a)

    # 리스트를 문자열로 : join 이용
    result1 = " ".join(result[0])

    for i in range(len(result)) :
        result[i] = " ".join(result[i])

    return result

In [ ]:
result = stop_word(df)
df.rename(columns = {'E':'지사','I':'VOC유형'},inplace=True)
data = df[['지사', 'VOC유형']]
data['sentence'] = result

In [ ]:
data.to_csv('../dataset/CAI_data_007.csv', index= False)